In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
from matplotlib import pyplot
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", 
             "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", 
             "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", 
             "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", 
             "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", 
             "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", 
             "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", 
             "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", 
             "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", 
             "some", "such", "no", "nor", "not", "only", "own", 
             "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
index_train = train.shape[0]-1
index_train

In [ ]:
# concatenate data for tfidf vectorizer
data = pd.concat([train[['qid','question_text']],test],ignore_index = True)

y_train = train.iloc[:,2]

In [ ]:
### Check split points are ok 

print(train.iloc[1306121,1] == data.iloc[1306121,1])
print(test.iloc[0,1] == data.iloc[1306122,1])

In [ ]:
data.head()

## Vectorising the data


In [ ]:
def clean_text_tfidf(text):
    
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text_tfidf)

In [ ]:
data_tfidf = tfidf_vect.fit_transform(data['question_text'])

In [ ]:
train_tfidf = data_tfidf[:index_train+1,:]

## Fitting an XGBoost Model

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
xgb = XGBClassifier()

In [ ]:
xgb.fit(train_tfidf,y_train)

## Predicting on test set

In [ ]:
test_tfidf = data_tfidf[index_train+1:,:]

In [ ]:
pred = xgb.predict(test_tfidf)

In [ ]:
len(pred)

In [ ]:
test.shape

In [ ]:
out_df = pd.DataFrame({"qid":test["qid"].values})
out_df['prediction'] = pred
out_df.to_csv("submission.csv", index=False)